In [35]:
# Kyle Dale
# CS-340-Client/Server Dev
# Professor Sanford
# Project Two
# 8/12-8/13/2023

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import Crud module and Animalhelter class
from Crud_Functionality_Module_KD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Input necessary authentication variables
USER = "aacuser"
PASS = "CS340UserPass"
HOST = 'nv-desktop-services.apporto.com'
PORT = 32072
DB = 'AAC'
COL = 'animals'

#db = AnimalShelter(username, password)
db = AnimalShelter(USER, PASS, HOST, PORT, DB, COL)

# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

# DEBUG
#print(df)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Project Two - KD'))),
    html.Hr(),
    #Display centered Grazioso Salvare Logo.
    html.Div(
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'display': 'block', 'margin': '0 auto'}
        ),
        style={'text-align': 'center'}
    ),
    
    html.Div(className='buttonRow', 
            style={'display' : 'flex'},
                children=[
                    html.Button(id='submit-button-one', n_clicks=0, children='Water'),
                    html.Button(id='submit-button-two', n_clicks=0, children='Mountain/Wilderness'),
                    html.Button(id='submit-button-three', n_clicks=0, children='Disaster/Individual'),
                    html.Button(id='submit-button-four', n_clicks=0, children='Unfilter')
                ]),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                                  for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10,
                         style_table={'width': '100%', 'maxWidth': '1000px'}
                        ),
    
    html.Br(),
    html.Hr(),
    html.Div([
        html.Div(id='map-id', className='col s12 m6'),
        dcc.Graph(id="piechart"),
    ], style={'display': 'flex', 'justify-content': 'space-between'}),
    dcc.Markdown("Project2KD"),
    dcc.Markdown("( ͡°( ͡° ͜ʖ( ͡° ͜ʖ ͡°)ʖ ͡°) ͡°)")
])


#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', "data"),
    [Input('submit-button-one', 'n_clicks'),
     Input('submit-button-two', 'n_clicks'),
     Input('submit-button-three', 'n_clicks'),
     Input('submit-button-four', 'n_clicks')]
)
def on_click(button1, button2, button3, button4):
    # Start case
    df = pd.DataFrame.from_records(db.read({}))
    
    if int(button1) > int(button2) and int(button1) > int(button3):
        water_species = ['Dog']
        water_breeds = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
        water_gender = 'Intact Female'
        water_age_range = (26, 156)
        
        df = df[df['animal_type'].isin(water_species)]
        df = df[df['breed'].isin(water_breeds)]
        df = df[df['sex_upon_outcome'] == water_gender]
        df = df[(df['age_upon_outcome_in_weeks'] >= water_age_range[0]) & (df['age_upon_outcome_in_weeks'] <= water_age_range[1])]
        
    elif int(button2) > int(button1) and int(button2) > int(button3):
        mw_species = ['Dog']
        mw_breeds = ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
        mw_gender = 'Intact Male'
        mw_age_range = (26, 156)
        
        df = df[df['animal_type'].isin(mw_species)]
        df = df[df['breed'].isin(mw_breeds)]
        df = df[df['sex_upon_outcome'] == mw_gender]
        df = df[(df['age_upon_outcome_in_weeks'] >= mw_age_range[0]) & (df['age_upon_outcome_in_weeks'] <= mw_age_range[1])]
        
    elif int(button3) > int(button1) and int(button3) > int(button2):
        di_species = ['Dog']
        di_breeds = ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
        di_gender = 'Intact Male'
        di_age_range = (20, 300)
        
        df = df[df['animal_type'].isin(di_species)]
        df = df[df['breed'].isin(di_breeds)]
        df = df[df['sex_upon_outcome'] == di_gender]
        df = df[(df['age_upon_outcome_in_weeks'] >= di_age_range[0]) & (df['age_upon_outcome_in_weeks'] <= di_age_range[1])]
        
    elif int(button4) > int(button1) and int(button4) > int(button2) and int(button4) > int(button3):
        # Handle Unfilter button case
        df = pd.DataFrame.from_records(db.read({}))
    
    # Cleanup Mongo _id field
    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_map(viewData):
    if not viewData:
        # If no data is available, return an empty map
        return dl.Map(style={'width': '1000px', 'height': '500px'},
                      center=[30.75, -97.48], zoom=10, children=[dl.TileLayer(id="base-layer-id")])

    dff = pd.DataFrame.from_dict(viewData)
    
    dff_subset = dff.head(10)  # Limit to 10 rows
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   *[
                       dl.Marker(position=[row['location_lat'], row['location_long']],
                                 children=[
                                     dl.Tooltip(f"{row['name']} - {row['breed']} - {row['age_upon_outcome_in_weeks']} weeks"),
                                     dl.Popup([
                                         html.H1("Animal Information"),
                                         html.P(f"Name: {row['name']}"),
                                         html.P(f"Breed: {row['breed']}"),
                                         html.P(f"Age: {row['age_upon_outcome_in_weeks']} weeks")
                                     ])
                                 ])
                       for _, row in dff_subset.iterrows()
                   ]
               ])
    ]


@app.callback(
    Output('piechart', "figure"),
    [Input('datatable-id', 'data'),
     Input('datatable-id', "page_current"),
     Input('datatable-id', "page_size")])
def update_piechart(data, page_current, page_size):
    df = pd.DataFrame.from_dict(data)
    
    start_idx = page_current * page_size
    end_idx = (page_current + 1) * page_size
    
    df_subset = df.iloc[start_idx:end_idx].head(30)  # Limit to 30 rows on the current page
    fig = px.pie(df_subset, names='breed', title='Pie Chart - Breed Distribution')
    return fig

app.run_server(debug=True)

Dash app running on http://127.0.0.1:19873/
